<a href="https://colab.research.google.com/github/uyangas/Visualizations-in-Python/blob/main/AirBnB_new_user_booking_destination_plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AirBnB new user booking destination (Plotly)

https://www.kaggle.com/c/airbnb-recruiting-new-user-bookings/data


`train_users.csv` - the training set of users
- `id`: user id
- `date_account_created`: the date of account creation
- `timestamp_first_active`: timestamp of the first activity, note that it can be earlier than date_account_created or date_first_booking because a user can search before signing up
- `date_first_booking`: date of first booking
- `gender`
- `age`
- `signup_method`
- `signup_flow`: the page a user came to signup up from
- `language`: international language preference
- `affiliate_channel`: what kind of paid marketing
- `affiliate_provider`: where the marketing is e.g. google, craigslist, other
- `first_affiliate_tracked`: whats the first marketing the user interacted with before the signing up
- `signup_app`
- `first_device_type`
- `first_browser`

`country_destination`: this is the target variable you are to predict

`sessions.csv` - web sessions log for users

`user_id`: to be joined with the column 'id' in users table
- `action`
- `action_type`
- `action_detail`
- `device_type`
- `secs_elapsed`

`countries.csv` - summary statistics of destination countries in this dataset and their locations

`age_gender_bkts.csv` - summary statistics of users' age group, gender, country of destination
sample_submission.csv - correct format for submitting your predictions


In [ ]:
from google.colab import files
files.upload()

In [2]:
# create "kaggle" directory
!mkdir ~/.kaggle
# save kaggle.json to "kaggle" directory
! cp kaggle.json ~/.kaggle/
# change the permission of the file
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
# download the dataset
! kaggle competitions download -c airbnb-recruiting-new-user-bookings --force

 90% 53.0M/59.1M [00:02<00:00, 9.21MB/s]
100% 59.1M/59.1M [00:02<00:00, 21.6MB/s]
  0% 0.00/546 [00:00<?, ?B/s]
100% 546/546 [00:00<00:00, 645kB/s]
  0% 0.00/2.46k [00:00<?, ?B/s]
100% 2.46k/2.46k [00:00<00:00, 3.73MB/s]
  0% 0.00/478k [00:00<?, ?B/s]
100% 478k/478k [00:00<00:00, 65.8MB/s]
100% 4.07M/4.07M [00:00<00:00, 18.6MB/s]

  0% 0.00/1.03M [00:00<?, ?B/s]
100% 1.03M/1.03M [00:00<00:00, 68.3MB/s]


In [4]:
# create directory to unzip the data
! mkdir ~/.airbnb_booking

In [5]:
# unzip the files
! unzip countries.csv.zip -d airbnb_booking
! unzip train_users_2.csv.zip -d airbnb_booking
! unzip age_gender_bkts.csv.zip -d airbnb_booking
! unzip sessions.csv.zip -d airbnb_booking
! unzip test_users.csv.zip -d airbnb_booking

Archive:  countries.csv.zip
  inflating: airbnb_booking/countries.csv  
Archive:  train_users_2.csv.zip
  inflating: airbnb_booking/train_users_2.csv  
Archive:  age_gender_bkts.csv.zip
  inflating: airbnb_booking/age_gender_bkts.csv  
Archive:  sessions.csv.zip
  inflating: airbnb_booking/sessions.csv  
Archive:  test_users.csv.zip
  inflating: airbnb_booking/test_users.csv  


In [7]:
! pip install impyute

## 1. Import data

In [8]:
# import necessary packages
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from datetime import datetime
from impyute.imputation.cs import mice
from sklearn.preprocessing import LabelEncoder
import pickle

In [158]:
# load the datasets
countries = pd.read_csv("airbnb_booking/countries.csv")
train = pd.read_csv("airbnb_booking/train_users_2.csv")
test = pd.read_csv("airbnb_booking/test_users.csv")
age_gender = pd.read_csv("airbnb_booking/age_gender_bkts.csv")
sessions = pd.read_csv("airbnb_booking/sessions.csv")

In [255]:
# merge train and test datasets
test['country_destination'] = 'Test'
all = pd.concat([train, test], axis=0, ignore_index=True)

In [256]:
all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275547 entries, 0 to 275546
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       275547 non-null  object 
 1   date_account_created     275547 non-null  object 
 2   timestamp_first_active   275547 non-null  int64  
 3   date_first_booking       88908 non-null   object 
 4   gender                   275547 non-null  object 
 5   age                      158681 non-null  float64
 6   signup_method            275547 non-null  object 
 7   signup_flow              275547 non-null  int64  
 8   language                 275547 non-null  object 
 9   affiliate_channel        275547 non-null  object 
 10  affiliate_provider       275547 non-null  object 
 11  first_affiliate_tracked  269462 non-null  object 
 12  signup_app               275547 non-null  object 
 13  first_device_type        275547 non-null  object 
 14  firs

In [257]:
# convert datetime
all['date_account_created'] = pd.to_datetime(all['date_account_created'])
all['date_first_booking'] = pd.to_datetime(all['date_first_booking'])
all['timestamp_first_active'] = pd.to_datetime(all['timestamp_first_active'], format='%Y%m%d%H%M%S')

In [258]:
print(f"Total number of users: {all.shape[0]}, Number of variables: {all.shape[1]}")

Total number of users: 275547, Number of variables: 16


In [259]:
all.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,NaT,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,NaT,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [260]:
for col in ['gender','signup_method','signup_flow','language','affiliate_channel','affiliate_provider','signup_app','first_device_type','first_browser','country_destination']:
  all[col] = all[col].astype('category')

## 2. Handling missing values

In [261]:
# plot the percentage of NA values in each column
fig = px.bar(x='Columns', 
             y='value', 
             data_frame=(all.isna().sum()/all.shape[0])\
             .reset_index()\
             .assign(Non_NA = lambda x: 1-x[0])\
             .rename({'index':'Columns',0:'NA_values'},axis=1)\
             .melt('Columns',['NA_values', 'Non_NA']),
             color='variable',
             orientation='v',
             category_orders={"Columns": ["NA values", "Non NA values"]},
             color_discrete_sequence=["rgb(0,184,184)", "rgb(227,227,227)"],
             hover_name='Columns',
             title='Percentage of NA values')

fig.update_layout(width=1000, height=400,
                  yaxis={'title':'Percentage'},
                  xaxis={'title':''},
                  plot_bgcolor='#fff')
fig.show()

`date_first_booking`, `age` and `first_affiliate_tracked` columns contained `67.7%`,  `42.4%` and `2.2%` NA values respectively.

In [262]:
# explore NA values of date_first_booking
all[all.date_first_booking.isna()][['id','date_account_created','timestamp_first_active','date_first_booking','country_destination']].head(10)

,id,date_account_created,timestamp_first_active,date_first_booking,country_destination
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,NaT,NDF
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,NaT,NDF
11,om1ss59ys8,2010-01-05,2010-01-05 05:18:12,NaT,NDF
13,dy3rgx56cu,2010-01-05,2010-01-05 08:32:59,NaT,NDF
14,ju3h98ch3w,2010-01-07,2010-01-07 05:58:20,NaT,NDF
16,2dwbwkx056,2010-01-07,2010-01-07 21:51:25,NaT,NDF
18,cxlg85pg1r,2010-01-08,2010-01-08 01:56:41,NaT,NDF
23,jha93x042q,2010-01-11,2010-01-11 22:40:15,NaT,NDF
24,7i49vnuav6,2010-01-11,2010-01-11 23:08:08,NaT,NDF
26,bjg0m5otl3,2010-01-12,2010-01-12 15:54:20,NaT,NDF


In [263]:
# count the destination countries
all[all.date_first_booking.isna()]['country_destination'].value_counts()

NDF      124543
Test      62096
other         0
US            0
PT            0
NL            0
IT            0
GB            0
FR            0
ES            0
DE            0
CA            0
AU            0
Name: country_destination, dtype: int64

To explore more about NA values of `date_first_booking` column. When `date_first_booking` is null, the person didn't place any booking. It seems it is not necessary to impute this value because imputation would alter the meaning of this variable.

Let's look at `age` column.

In [264]:
# descriptive statistics of age
all.age.describe()

count    158681.000000
mean         47.145310
std         142.629468
min           1.000000
25%          28.000000
50%          33.000000
75%          42.000000
max        2014.000000
Name: age, dtype: float64

The maximum value that the `age` column contains is 2014 which could be a typo or an error because it is unlikely that people can live up to over 2000 years, yet alone over 100 years. The minimum value of `age` is 1 which is highly unlikely as well.

In [265]:
# age greater than 100
all['age'][all.age>100].value_counts().head(10)

105.0     1351
2014.0     710
110.0      228
104.0       52
101.0       40
102.0       39
2013.0      39
109.0       36
103.0       30
107.0       28
Name: age, dtype: int64

In [266]:
pio.templates.default = "ggplot2"
# create subplots
fig = make_subplots(1, 2, 
                    subplot_titles=["Histogram of Age", 'Boxplot of Age'],
                    column_widths=[0.3, 0.3])
# add histogram trace to the subplot
fig.add_trace(go.Histogram(x=all[all.age<110].age, 
                           marker_color='rgb(0,186,202)'), 
              1,1)
# add boxplot trace to the subplot
fig.add_trace(go.Box(x=all[all.age<110].age, marker_color='rgb(0,189,166)'), 1,2)
# update the figure size
fig.update_layout(width=1200, height=400, showlegend=False)
fig.show()

From the plots above, `age` has several values less than 10 which could most likely errors. Therefore, extreme values i.e. `age` less than 10 and greater than 100 are considered as outliers, removed and filled with imputation.

In [267]:
# remove extreme values and add np.nan
all['age'][(all['age'] <10)|(all['age'] >100)] = np.nan

Since `age` is continuous variable and filling `NA` with mean or median value would cause distribution change, `MICE` method was used. To use `MICE` all categorical values must be numerical. Therefore, the values of caterical attributes were encoded using `LabelEncoder`.

In [268]:
# create a copy of all dataframe
all_transformed = all.copy()
# encode the values of each categorical attribute
for col in ['gender','signup_method','signup_flow','language','affiliate_channel',
            'affiliate_provider','signup_app','first_device_type','country_destination','first_browser']:
  # call the object
  le = LabelEncoder()
  # fit to the column
  le.fit(all_transformed[col])
  # transform the values
  all_transformed[col] = le.transform(all_transformed[col])

In [269]:
all_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275547 entries, 0 to 275546
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   id                       275547 non-null  object        
 1   date_account_created     275547 non-null  datetime64[ns]
 2   timestamp_first_active   275547 non-null  datetime64[ns]
 3   date_first_booking       88908 non-null   datetime64[ns]
 4   gender                   275547 non-null  int64         
 5   age                      155932 non-null  float64       
 6   signup_method            275547 non-null  int64         
 7   signup_flow              275547 non-null  int64         
 8   language                 275547 non-null  int64         
 9   affiliate_channel        275547 non-null  int64         
 10  affiliate_provider       275547 non-null  int64         
 11  first_affiliate_tracked  269462 non-null  object        
 12  signup_app      

In [270]:
# use MICE to impute age variable
# imputed = mice(all_transformed.drop(['id',
#                                      'country_destination',
#                                      'date_first_booking',
#                                      'date_account_created',
#                                      'timestamp_first_active',
#                                      'first_affiliate_tracked'],axis=1).values)
# imputed[:,1]

# save imputed age variable as pickle 
# with open('age.pkl','wb') as f:
#   pickle.dump(imputed[:,1], f)

# running MICE takes long time, saved age array is loaded to use
imputed_age = pickle.load(open('age.pkl','rb'))

In [271]:
# replace NA values with imputed values
all['age'] = imputed_age.astype('int')
# delete unnecessary dataframes
del all_transformed

In [273]:
all.age.describe()

count    275547.000000
mean         35.733047
std           8.919339
min          15.000000
25%          31.000000
50%          35.000000
75%          38.000000
max         100.000000
Name: age, dtype: float64

Now, let's look at `first_affiliate_tracked` column.

In [274]:
print(f"Unique values: {all['first_affiliate_tracked'].unique()}")
print(f"Number of NA values: {all['first_affiliate_tracked'].isna().sum()}")

Unique values: ['untracked' 'omg' nan 'linked' 'tracked-other' 'product' 'marketing'
 'local ops']
Number of NA values: 6085


In [275]:
# dataframe with value counts
fat_count = pd.concat([all[['id','first_affiliate_tracked']].groupby('first_affiliate_tracked').count().reset_index().sort_values('id'), 
                       pd.DataFrame({'first_affiliate_tracked':['nan'], 'id':[all.first_affiliate_tracked.isna().sum()]})], axis=0).sort_values('id')

# pie chart
fig = go.Figure()

colors = ['rgb(140,140,140)',
          'rgb(169,169,169)',
          'rgb(120,120,120)',
          'rgb(0,186,202)',
          'rgb(192,192,192)',
          'rgb(203,203,203)',
          'rgb(130,130,130)',
          'rgb(226,226,226)']

fig.add_trace(go.Pie(labels=fat_count['first_affiliate_tracked'],
                     values=fat_count['id'],
                     hole=.3,
                     rotation=260,
                     textinfo='label+percent',
                     showlegend=False,
             ))

fig.update_layout(width=450, height=450, title="Precentage of values in first_affiliate_tracked")
fig.update_traces(marker=dict(colors=colors))
fig.show()

`first_affiliate_tracked` column has only 6,085 `NA` values i.e. 2.21% of the total number of rows. Based on the number of categories and the number of values in each category, `NA` values were treated as a separate category and filled as `No Value`.

In [276]:
# filled missing values with No Value instead of other types of filling methods
all['first_affiliate_tracked'] = all['first_affiliate_tracked'].astype('str').fillna('No Value').astype('category')

In [277]:
all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275547 entries, 0 to 275546
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   id                       275547 non-null  object        
 1   date_account_created     275547 non-null  datetime64[ns]
 2   timestamp_first_active   275547 non-null  datetime64[ns]
 3   date_first_booking       88908 non-null   datetime64[ns]
 4   gender                   275547 non-null  category      
 5   age                      275547 non-null  int64         
 6   signup_method            275547 non-null  category      
 7   signup_flow              275547 non-null  category      
 8   language                 275547 non-null  category      
 9   affiliate_channel        275547 non-null  category      
 10  affiliate_provider       275547 non-null  category      
 11  first_affiliate_tracked  275547 non-null  category      
 12  signup_app      

## 3. Visualization

In [278]:
fig = go.Figure()
# convert first active timestamp to YYYYMMDD format
all['timestamp_first_active_2'] = [date.strftime('%Y-%m-%d') for date in all['timestamp_first_active']]
# count number of people by date
df = all[['id','date_account_created']].groupby('date_account_created').count().reset_index()
df1 = all[['id','timestamp_first_active_2']].groupby('timestamp_first_active_2').count().reset_index()
df2 = all[['id','date_first_booking']].groupby('date_first_booking').count().reset_index()
# add line graph to fig
fig.add_trace(go.Scatter(x=df['date_account_created'],
                         y=df['id'],
                         name='date_account_created'))
fig.add_trace(go.Scatter(x=df1['timestamp_first_active_2'],
                         y=df1['id'],
                         name='timestamp_first_active'))
fig.add_trace(go.Scatter(x=df2['date_first_booking'],
                         y=df2['id'],
                         opacity=0.6,
                         name='date_first_booking'))
# update figure
fig.update_layout(title='Number of people who created accounts and first booking',
                  width=1300, height=500)
fig.show()

The daily number of created accounts  increased steadily until Jul 2014 and decreased since then. As for the daily number of people who placed a booking for the first time increased until Jun 2014 and declined since then. This decline could be due to many reasons. Let's look at how many days after creating an account, people place the first booking.

In [294]:
# create datetime variables of YYYY-MM format
all['date_created'] = [date.strftime('%Y-%m') for date in all['date_account_created']]
all['date_active'] = [date.strftime('%Y-%m') for date in all['timestamp_first_active']]
all['date_booking'] = ['NA' if pd.isnull(date) else date.strftime('%Y-%m') for date in all['date_first_booking']]

In [298]:
all.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,timestamp_first_active_2,date_created,date_active,date_booking
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,NaT,-unknown-,33,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,2009-03-19,2010-06,2009-03,NA
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,NaT,MALE,38,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,2009-05-23,2011-05,2009-05,NA
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,2010-08-02,FEMALE,56,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,2009-06-09,2010-09,2009-06,2010-08
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,2012-09-08,FEMALE,42,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,2009-10-31,2011-12,2009-10,2012-09
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,2010-02-18,-unknown-,41,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,2009-12-08,2010-09,2009-12,2010-02


In [305]:
sank = pd.concat([all[['id','date_created','date_active']]\
            .groupby(['date_created','date_active'])\
            .count()\
            .reset_index()\
            .rename({'date_created':'label1','date_active':'label2','id':'value'},axis=1)\
            .assign(),
          all[['id','date_created','date_booking']]\
            .groupby(['date_created','date_booking'])\
            .count()\
            .reset_index()\
            .rename({'date_created':'source','date_booking':'target','id':'value'},axis=1)], 
          axis=0)

In [306]:
sank.source.unique()

array(['2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2010-06',
       '2010-07', '2010-08', '2010-09', '2010-10', '2010-11', '2010-12',
       '2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06',
       '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12',
       '2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2012-06',
       '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2012-12',
       '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06',
       '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12',
       '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06',
       '2014-07', '2014-08', '2014-09'], dtype=object)

In [307]:
sank.target.unique()

array(['2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2009-03',
       '2010-06', '2010-07', '2010-08', '2009-06', '2009-12', '2010-09',
       '2010-10', '2010-11', '2010-12', '2011-01', '2011-02', '2011-03',
       '2011-04', '2009-05', '2011-05', '2011-06', '2011-07', '2011-08',
       '2011-09', '2011-10', '2011-11', '2009-10', '2011-12', '2012-01',
       '2012-02', '2012-03', '2012-04', '2012-05', '2012-06', '2012-07',
       '2012-08', '2012-09', '2012-10', '2012-11', '2012-12', '2013-01',
       '2013-02', '2013-03', '2013-04', '2013-05', '2013-06', '2013-07',
       '2013-08', '2013-09', '2013-10', '2013-11', '2013-12', '2014-01',
       '2014-02', '2014-03', '2014-04', '2014-05', '2014-06', '2014-07',
       '2014-08', '2014-09', 'NA', '2014-10', '2014-11', '2014-12',
       '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06'],
      dtype=object)

In [293]:
fig = go.Figure()

fig.add_trace(go.Sankey())

dtype('O')

In [202]:
# function that visualizes datetime variable

def viz_date(df, col, col2=None):
  df['year'] = [val.year for val in df[col]]
  df['month'] = [val.strftime("%m") for val in df[col]]
  df['day'] = [val.weekday() for val in df[col]]
  df['month_day'] = df['month'].astype('str') + "_" + (df['day']+1).astype('str')
  number_year = df.year.unique()

  fig = go.Figure()

  for year in number_year:
    df_sub = df[df.year==year][['id', 'year','day']].groupby(['year','day']).count().reset_index()

    fig.add_trace(go.Scatter(x=df_sub['day'], 
                             y=df_sub['id'], 
                             mode='lines',
                             name=str(year)
                             )
    )
  
  fig.update_layout(width=1500, height=500)
  fig.update_xaxes()
  return fig.show()

In [226]:
all['days_created_active'] = all.date_account_created - all.timestamp_first_active
all['days_booking_created'] = all.date_first_booking - all.date_account_created